In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from IPython.core.display import display, HTML
import yaml
import pdb

In [2]:
telescopes = dict.fromkeys(['CHLAT_20210401.yaml', 'SPLAT_20210401.yaml', 'SAT_pole_20210401.yaml'])

yaml_file = list(telescopes.keys())[0]
dd = yaml.safe_load(open(yaml_file))
version_date = str(dd['version']['date'])
version_telescope = str(dd['version']['name'])
del dd['version']

ch_names = list(dd['instrument']['camera_config']['elements']['cam_1']['chan_config']['elements'].keys())

#fake values

in_param = 'pwv'
out_param = 'NET'
param_vec = np.arange(500,8000,100)
base_value = 1000
nominal = {}
nominal = dd['instrument'][in_param]
#change output
output = param_vec**2

In [3]:
def display_side_by_side(dfs:list, captions:list):
    """Display tables side by side to save vertical space
    Input:
        dfs: list of pandas.DataFrame
        captions: list of table captions
    """
    output = ""
    combined = dict(zip(captions, dfs))
    for caption, df in combined.items():
        output += df.style.set_table_attributes("style='display:inline'").set_caption(caption)._repr_html_()
        output += "\xa0\xa0\xa0"
    display(HTML(output))

In [4]:
def NET_TeleTable(telescope, nominal, percent):
    """
    telescope is dictionary of in, out, for each channel
    """
    
    files = ['SAT_pole_20210401', 'SPLAT_20210401', 'CHLAT_20210401']
    yaml_file, dd, nominal, top, tabs, Net = \
    {},{},{},{},{},{}
    yaml_file =  f'{telescope}_20210401.yaml'
    #dd = yaml.safe_load(open(yaml_file))
    #del dd['version']
    #ch_names = list(dd['instrument']['camera_config']['elements']['cam_1']['chan_config']['elements'].keys())
    #nominal = {}
    #nominal = dd['instrument']['channel_default'][in_param]
    outputs[out_param] = {}
    for chan in ch_names:
        outputs[out_param][chan] = [] 
    ypv,left_yv,right_yv,slope,NET,ynv,\
    left,right,pxvp,nxvp,label,left_slope,right_slope = {},{},{},{},{},{},{},{},{},{},{},{},{}
    vals, data = [],[]   
    for chan in ch_names:
        Net[chan] = outputs[out_param][chan]
        ynv[chan] = np.interp(nominal,param_vec,Net[chan])#y-value for nominal
        left = np.where(param_vec<nominal)[0] #left of nominal, holds index locations
        left_xv = param_vec[left]
        left_yv[chan] = Net[chan][left]  #The corresponding y-values
        right = np.where(param_vec>nominal)[0] #right of and including nominal, holds index locations
        right_xv = param_vec[right]
        right_yv[chan] = Net[chan][right] #The corresponding y-values
        left_slope[chan] = (ynv[chan]-left_yv[chan][-1])/(nominal-left_xv[-1]) #slope from the left
        right_slope[chan] = (right_yv[chan][0]-ynv[chan])/(right_xv[0]-nominal) #slope from the right
        ypv[chan] = {}
        pxvp[chan] = {}
        nxvp[chan] = {}
        slope[chan] = {}
        step = 1e-6 #step size in finding slope
        slope[chan] = (left_slope[chan]+right_slope[chan])/2
#         slope[chan] = ( np.interp((nominal+step),right_xv[chan],right_yv[chan]) - ynv[chan])/((nominal[chan]+step) - nominal[chan])
        vals.append([])
        for per in percent:
            ypv[chan][per] = ynv[chan]*((100+per)/100) #y values for each positive
                                                      #percent for every aspect of output
            pxvp[chan][per] = np.interp(ypv[chan][per], right_yv[chan],right_xv)#right_xv[chan][np.where(right_yv[chan] >= ypv[chan][per])][0] #the corresponding pos values in x
            nxvp[chan][per] = np.interp(ypv[chan][per], np.flip(left_yv[chan]),np.flip(left_xv))#left_xv[chan][np.where( left_yv[chan] >= ypv[chan][per])][-1] #the corresponding neg values in x
#       The if loops make it so that if the x values are out of range of the input range, they say 'NaN'
            if ((nxvp[chan][per] == start) or (nxvp[chan][per] < start) or (nxvp[chan][per] > end) or (nxvp[chan][per]==end)) :
                nxvp[chan][per] = 'NaN' #Out of range
            if ((pxvp[chan][per] == start) or (pxvp[chan][per] < start) or (pxvp[chan][per] > end) or (pxvp[chan][per]==end)) :
                pxvp[chan][per] = 'NaN' #Out of range 
    for k in range(len(ch_names)):
        NET[ch_names[k]] = {}
        for p in range(len(percent)):
            label[p] = f'neg_{percent[len(percent)-p-1]}' #Label for the table 
                                                           #This part does the negative
            NET[ch_names[k]][label[p]] = nxvp[ch_names[k]][percent[-1-p]] #NET dictionary for output
            vals[k].append(nxvp[ch_names[k]][percent[-1-p]]) #Values for the table
    #This part does the nominal
    label[len(percent)] = 'nominal'
    label[len(percent)+1] = 'slope'
    #pdb.set_trace()
    for k in range(len(ch_names)):
        NET[ch_names[k]][label[len(percent)]] = nominal
        NET[ch_names[k]][label[len(percent)+1]] = slope[ch_names[k]]
        vals[k].append(nominal)
        vals[k].append(slope[ch_names[k]])
        for p in range(len(percent)):
            label[len(percent)+2+p] = f'pos_{percent[p]}'
            NET[ch_names[k]][label[len(percent)+2+p]] = pxvp[ch_names[k]][percent[p]]
            vals[k].append(pxvp[ch_names[k]][percent[p]])
        title = [] #matrix of the label to put in the table
        row = []
        for l in range(len(label)):
            title.append(label[l])
            row.append(vals[k][l])
        data.append(row)
    #pdb.set_trace()
    df = pd.DataFrame(data,index=ch_names, columns=title)
    return df #, NET#ynv,left_xv,left_yv,right_xv,right_yv#df, NET

In [5]:
npts = len(param_vec)
param_dict = {}
in_put = ['Tc','carrier_index', 'psat_factor']
start = [0.14,0,1.5]
end = [0.2,4,4]

# {'startval':0.14,'endval':0.2,'npts':npts,'nominal':}

for i in range(len(in_put)):
    param_dict[in_put[i]]= {}
    for s in range(len(start)):
        param_dict[in_put[i]]['startval']= start[i]
        param_dict[in_put[i]]['endval']= end[i]
        param_dict[in_put[i]]['npts']= npts

param_dict

{'Tc': {'startval': 0.14, 'endval': 0.2, 'npts': 75},
 'carrier_index': {'startval': 0, 'endval': 4, 'npts': 75},
 'psat_factor': {'startval': 1.5, 'endval': 4, 'npts': 75}}

In [6]:
%%time
pnts = 50

teles = ['SAT_pole', 'SPLAT','CHLAT']
#in_put = ['Tc','carrier_index', 'psat_factor'] #Look in the yaml files
percents = [0.3,0.5,1]
pd_frame = []
# dfn = []
# dfpsat = []

for i in range(len(teles)):
    pd_frame.append([])
    pd_frame[i] = NET_TeleTable(teles[i], in_param, pnts, percents)[0] #Tc: 0.14-0.2


TypeError: NET_TeleTable() takes 3 positional arguments but 4 were given

In [7]:
ch_names

['LF_1', 'LF_2', 'MF_1', 'MF_2', 'UHF_1', 'UHF_2']